In [1]:
using Pipe
using Iterators
using DataStructures

In [47]:
function loadphrases(filename)
    (@pipe readall(filename) |> split(_,'\n'))[1:end-1];
end

loadphrases (generic function with 1 method)

In [68]:
phrase_range = 14380:20000  #old testerment14380:40069
ps = readcsv("results/gutten.csv")[phrase_range,:]
phrases = loadphrases("results/gutten.phrases.txt") [phrase_range]

5621-element Array{SubString{ASCIIString},1}:
 "the old testament of the king *UNKNOWN* bible"                                                                                                                                                                                      
 "the first book of *UNKNOWN* : called *UNKNOWN*"                                                                                                                                                                                     
 "1 : 1 in the beginning god created the heaven and the earth ."                                                                                                                                                                      
 "1 : 2 and the earth was without form , and void ; and darkness was upon the face of the deep ."                                                                                                                                     
 "and the spirit of god moved 

In [61]:
ps1 = readcsv("results/language_simple_wikipedia.csv")
phrases1 = loadphrases("results/language_simple_wikipedia.phrases.txt")
ps2 = readcsv("results/boy_destroyed_room.csv")
phrases2 = loadphrases("results/boy_destroyed_room.txt")

ps = [ps1;ps2]
phrases = [phrases1,phrases2]

69-element Array{SubString{ASCIIString},1}:
 "language is the normal way humans communicate ."                                                                            
 "only humans use language , though many other animals communicate in various ways ."                                         
 "human language has *UNKNOWN* , a set of rules for connecting words together to make statements and question ."              
 "language can also be changed , by adding new words , for example , to describe new things ."                                
 "other animals may inherit a set of calls which have pre-set functions ."                                                    
 "language may be done by speech or by writing or by moving the hands to make signs ."                                        
 "it follows that language is not just any way of communicating ."                                                            
 "even some human communication is not language : see *UNKNOWN* com

In [3]:
using PyCall
@pyimport sklearn
@pyimport sklearn.mixture as sklm

In [ ]:
gmm=sklm.GMM(n_components=4, covariance_type="tied", n_init=25) 
gmm[:fit](ps)
gmm[:weights_]'

In [ ]:
function show_clusters_gmm(gmm,xs, words)
    scores = gmm[:score](xs)
    order = sort([1:length(scores);], by=ii->scores[ii], rev=true)
    clusters = DefaultDict(Int, Vector{String}, ()->String[])
    for ii in order
        word = words[ii]
        cluster =  gmm[:predict](xs[ii,:])[1]
        push!(clusters[cluster], word)
    end
    
    for cluster in keys(clusters)
        println("** $cluster **")
        for word in clusters[cluster]
            println(" - ", word)
        end
        println()
    end
    clusters
end
clusters = show_clusters_gmm(gmm, ps, phrases)

In [4]:
@pyimport sklearn.cluster as skl_cluster

In [5]:
bandwidth = skl_cluster.estimate_bandwidth(ps, quantile=0.05)
ms = skl_cluster.MeanShift(bandwidth=bandwidth, bin_seeding=false, min_bin_freq=1)

println(ms[:fit](ps))
println( ms[:labels_] |> unique |> sort)

PyObject MeanShift(bandwidth=1.04845273746, bin_seeding=False, cluster_all=True,
     min_bin_freq=1, seeds=None)
[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]


In [6]:
function show_clusters(model,words, labels)
    clusters = DefaultDict(Int, Vector{String}, ()->String[])
    for ii in 1:length(words)
        word = words[ii]
        cluster =  labels[ii]
        push!(clusters[cluster], word)
    end
    
    for cluster in keys(clusters)
        println("** $cluster **")
        for word in clusters[cluster]
            println(" - ", word)
        end
        println()
    end
    clusters
end

show_clusters(ms,phrases, ms[:labels_])

** 18 **
 - mathematics and computer science use created languages called formal languages *UNKNOWN* like computer programming languages *UNKNOWN* , but these may or may not be *UNKNOWN* true ' languages .

** 2 **
 - *UNKNOWN* is often called `` the international language '' .
 - *UNKNOWN* as a first language : 380 million .
 - *UNKNOWN* taught as a second language , but with no official status : anyone 's guess , up to 1000 million .

** 16 **
 - sentences with two types of expression : *UNKNOWN* and *UNKNOWN* : *UNKNOWN* is here .

** 11 **
 - *UNKNOWN* words are there to make the language work : and , not , in , out , what , etc. .

** 21 **
 - so if they learn a second language , it is difficult , and they have little chance of speaking it as well as their native language .

** 0 **
 - language is the normal way humans communicate .
 - only humans use language , though many other animals communicate in various ways .
 - human language has *UNKNOWN* , a set of rules for connecting 

DefaultDict{Int64,Array{String,1},Function} with 23 entries:
  18 => String["mathematics and computer science use created languages called f…
  2  => String["*UNKNOWN* is often called `` the international language '' .","…
  16 => String["sentences with two types of expression : *UNKNOWN* and *UNKNOWN…
  11 => String["*UNKNOWN* words are there to make the language work : and , not…
  21 => String["so if they learn a second language , it is difficult , and they…
  0  => String["language is the normal way humans communicate .","only humans u…
  7  => String["a type of school subject . *UNKNOWN* says that *UNKNOWN* langua…
  9  => String["before the invention of recording , the writing system was the …
  10 => String["dummy elements : *UNKNOWN* likes to swim , so do *UNKNOWN*"]
  19 => String["types of language"]
  17 => String["in most languages , *UNKNOWN* being one , the order of the word…
  8  => String["ways of linking : sink or swim ."]
  22 => String["all the basic sound units , or

In [10]:
using Distances
push!(LOAD_PATH, "../word-embeddings2")
import WordEmbeddings.cosine_sim

In [66]:
pps = ps'
#cosine_sim = Float64[WordEmbeddings.cosine_sim(pps[:,ii],pps[:,jj]) for ii in 1:size(pps,2), jj in 1:size(pps,2)];
#dot_sim = -1*Float64[dot(pps[:,ii],pps[:,jj]) for ii in 1:size(pps,2), jj in 1:size(pps,2)];
#cosine_sim2 = -pairwise(CosineDist(),pps)
euclidean_sim = -pairwise(Euclidean(),pps)


25690x25690 Array{Float64,2}:
 -0.0      -1.80406  -1.86725   -1.8686    …  -1.90203  -1.75395  -1.87132 
 -1.80406  -0.0      -1.63378   -1.62419      -1.6927   -1.70688  -1.65505 
 -1.86725  -1.63378  -0.0       -0.270393     -1.69574  -1.4128   -0.80709 
 -1.8686   -1.62419  -0.270393  -0.0          -1.70449  -1.42439  -0.78637 
 -1.8859   -1.57007  -1.14471   -1.18153      -1.82579  -1.47787  -1.37767 
 -1.85724  -1.61298  -0.297991  -0.207416  …  -1.70366  -1.42179  -0.80646 
 -1.84496  -1.61635  -0.291761  -0.182719     -1.72513  -1.43139  -0.809711
 -1.76018  -1.93893  -1.46953   -1.50954      -1.6604   -1.40131  -1.49624 
 -1.87078  -1.54928  -1.156     -1.15035      -1.70995  -1.3819   -1.33617 
 -1.83997  -1.90743  -1.48875   -1.52306      -1.87711  -1.56265  -1.56827 
 -1.85444  -1.62215  -0.321569  -0.161278  …  -1.70684  -1.42338  -0.815314
 -1.78053  -1.94427  -1.5171    -1.5713       -1.77498  -1.50259  -1.65424 
 -1.87081  -1.54766  -1.15402   -1.15208      -1.70318  -1

In [67]:
affprop = skl_cluster.AffinityPropagation(damping=0.5)
affprop[:fit_predict](euclidean_sim)
centers = affprop[:cluster_centers_indices_]+1 # change to julia indexing
for phrase in phrases[centers]
    println(phrase)
end

LoadError: PyError (:PyObject_Call) <type 'exceptions.MemoryError'>
MemoryError()
  File "/usr/local/lib/python2.7/dist-packages/sklearn/base.py", line 343, in fit_predict
    self.fit(X)
  File "/usr/local/lib/python2.7/dist-packages/sklearn/cluster/affinity_propagation_.py", line 271, in fit
    self.affinity_matrix_ = -euclidean_distances(X, squared=True)
  File "/usr/local/lib/python2.7/dist-packages/sklearn/metrics/pairwise.py", line 157, in euclidean_distances
    XX = np.sum(X * X, axis=1)[:, np.newaxis]

while loading In[67], in expression starting on line 2

In [64]:
show_clusters(affprop,phrases, affprop[:labels_])

** 2 **
 - *UNKNOWN* of language
 - *UNKNOWN* is the language with the most native speakers in the world , but *UNKNOWN* is not really a language .
 - *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* : 390 million native speakers .

** 0 **
 - language is the normal way humans communicate .
 - only humans use language , though many other animals communicate in various ways .
 - language can also be changed , by adding new words , for example , to describe new things .
 - language may be done by speech or by writing or by moving the hands to make signs .
 - language is a word that may be used by extension : the language of a community or country .
 - grammar consists of studying how words fit together to mean something .
 - adults do not learn languages the same way they did when they were young .

** 7 **
 - a type of school subject . *UNKNOWN* says that *UNKNOWN* languages are at risk of becoming *UNKNOWN* .
 - before the invention of recording , the writing system was the only way to keep tra

DefaultDict{Int64,Array{String,1},Function} with 11 entries:
  2  => String["*UNKNOWN* of language","*UNKNOWN* is the language with the most…
  0  => String["language is the normal way humans communicate .","only humans u…
  7  => String["a type of school subject . *UNKNOWN* says that *UNKNOWN* langua…
  9  => String["*UNKNOWN*","*UNKNOWN*"]
  10 => String["the faculty of speech .","the sounds come out in a sequence , n…
  8  => String["*UNKNOWN* capacity to learn and use language is inherited .","*…
  6  => String["it follows that language is not just any way of communicating .…
  4  => String["all the basic sound units , or *UNKNOWN* , have this in common …
  3  => String["words *UNKNOWN* which may be made up of more than one *UNKNOWN*…
  5  => String["humans also use language for thinking .","all languages have :"…
  1  => String["human language has *UNKNOWN* , a set of rules for connecting wo…

In [ ]:
using Clustering

In [ ]:
affprop_j = affinityprop(similarities, damp=0.25)
#@assert(affprop_j.converged)
for ii in affprop_j.exemplars
    println(phrases[ii])
end

In [ ]:
using MultivariateStats

pca = fit(PCA, ps'; maxoutdim=2)


Y = transform(pca, ps')'


In [ ]:
using TSne
#Y = tsne(ps, no_dims = 2, initial_dims = -1, max_iter = 1000, perplexity = 30.0)
Y = tsne(ps, 10,  -1, 2000, 20.0)

In [ ]:
lbls = zeros(50)
lbls[end-5:end]=1
plot(x=Y[:,1], y=Y[:,2], color=lbls, label=phrases,Geom.point, Geom.label(position=:centered,hide_overlaps=false))

In [ ]:
using Gadfly
set_default_plot_size(20cm, 20cm)